In [115]:
from collections import deque
import logging

from numpy import exp, inf, log
from numpy.random import rand
import numpy as np

## Logging Config

In [116]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Random Variable Helpers

In [117]:
def gen_exp_rv(ld):
    return -1 / ld * log(1 - rand()) # using inverse CDF of exp RVs

def gen_exp_rv_hourly_rate(hours):
    return gen_exp_rv(hours) * 60 * 60

## Classes

### Stall
Handles queue and departure of customers

Including whether or not customer joins, according to this formula


Let $c \in [0, 1]$, where $c$ is a parameter of the simulation determining the willingness of a random customer to queue.

Let q be the queue length

\begin{align*}
\text{Queue joined} = \begin{cases} 
      \text{Joined} & \text{if }U(0, 1) \leq c^{q}\\
      \text{Not joined} & \text{o.w.}\\
   \end{cases}
\end{align*}

In [135]:
class Stall:
    def __init__(self, simulator, c, mu):
        self.c = c
        self.mu = mu
        self.simulator = simulator
        self.queue = deque([])
        self._departure_times = deque([])
        
    def add_customer(self, customer):
        def gen_departure_time():
            if not self._departure_times:
                return gen_exp_rv(self.mu) + self.simulator.time
            else:
                return gen_exp_rv(self.mu) + max(
                    self._departure_times[-1],
                    self.simulator.time
                )
                
        def is_customer_join():
            return rand() <= self.c**len(self.queue)

        if is_customer_join():
            self.queue.append(customer)
            departure_time = gen_departure_time()
            self._departure_times.append(departure_time)
        
    def serve_customer(self):
        if self.queue:
            self.queue.popleft()
            self._departure_times.popleft()
        else:
            raise IndexError('Queue is empty')
            
    @property
    def next_departure_time(self):
        if not self._departure_times:
            return inf
        return self._departure_times[0]

### Customer and Customer Generator
Handles arrival of customers

In [136]:
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time

# handles arrival of customers 
class CustomerGenerator:
    def __init__(self, simulator, ld):
        self.simulator = simulator
        self.ld = ld
        self._next_arrival_time = -1
        
    def generate_customer(self):
        return Customer(self.next_arrival_time)
        
    @property
    def next_arrival_time(self):
        if self.simulator.time >= self._next_arrival_time:
            next_arrival_time = gen_exp_rv(self.ld) + self.simulator.time
            self._next_arrival_time = next_arrival_time
            
        return self._next_arrival_time

### Tracker
Handles cycle tracking, and pretty printing

In [137]:
class Tracker:
    def __init__(self):
        self.info = {}
        
    def track(self, queue_length, time):
        if queue_length not in self.info:
            self.info[queue_length] = []
            
        self.info[queue_length].append(time)
        
    def __str__(self):
        res = []
        for queue_length in sorted(self.info.keys()):
            vals = self.info[queue_length]
            
            num_cycles = len(vals) - 1
            
            if num_cycles == 0:
                continue
                
            span = vals[-1] - vals[0]
            mean_cycle_length = span / num_cycles
            
            res.append(
                f'Queue Length {queue_length}. {num_cycles} Cycles. Mean Cycle Length: {mean_cycle_length:.2f}'
            )
            
        return '\n'.join(res)

### Simulator
Orchestrator

In [141]:
class Simulator:
    def __init__(self, c, mu, ld, maxtime=9999999):
        self.customer_generator = CustomerGenerator(self, ld)
        self.stall = Stall(self, c, mu)
        self.maxtime = maxtime
        self.tracker = Tracker()
        self.time = 0
        
    def update_time(self, time):
        self.time = time
        
    def track(self):
        self.tracker.track(
            len(self.stall.queue),
            self.time
        )
    
    def iterate(self):
        def handle_arrival():
            customer = self.customer_generator.generate_customer()
            self.stall.add_customer(customer)
            
        def handle_departure():
            self.stall.serve_customer()
            
        next_arrival_time, next_departure_time = self.customer_generator.next_arrival_time, self.stall.next_departure_time
        is_arrival_first = next_arrival_time < next_departure_time
        
        if is_arrival_first:
            self.update_time(next_arrival_time)
            handle_arrival()
            
            logger.debug(f'Handled arrival at {self.time:.3f}')
        else:
            handle_departure()
            self.update_time(next_departure_time)
            
            logger.debug(f'Handled departure at {self.time:.3f}')
            
        self.track()
        
    # useful for debugging
    def iterate_n(self, n):
        for _ in range(n):
            self.iterate()
            
    def run(self):
        while self.time <= self.maxtime:
            self.iterate()

In [177]:
mu = 3 # services per min
ld = 2 # arrivals per min
maxtime = 100000
s = Simulator(mu, ld, 0.5, maxtime)
# s.iterate_n(10)
s.run()
print(s.tracker)

Queue Length 0. 37474 Cycles. Mean Cycle Length: 2.67
Queue Length 1. 46934 Cycles. Mean Cycle Length: 2.13
Queue Length 2. 11849 Cycles. Mean Cycle Length: 8.44
Queue Length 3. 2954 Cycles. Mean Cycle Length: 33.82
Queue Length 4. 716 Cycles. Mean Cycle Length: 139.39
Queue Length 5. 193 Cycles. Mean Cycle Length: 487.64
Queue Length 6. 45 Cycles. Mean Cycle Length: 2009.44
Queue Length 7. 5 Cycles. Mean Cycle Length: 17357.77
